In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Step 1: Connect to the Sakila Database

# Your password (make sure to fill this in)
password = ''

# Create a connection to the Sakila database
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

# Step 2: Retrieve Rental Data for a Specific Month and Year

def rentals_month(engine, month, year):
    # SQL query to get rental data for the specified month and year
    query = f'''
    SELECT * 
    FROM rental 
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    '''
    # Execute the query and store the result in a DataFrame
    data = pd.read_sql_query(query, engine)
    return data

# Step 3: Count Rentals by Customer for a Specific Month and Year

def rental_count_month(data, month, year):
    # Group the data by customer and count the rentals
    rental_counts = data.groupby('customer_id').size().reset_index(name=f'rentals_{month}_{year}')
    return rental_counts

# Step 4: Compare Rentals Between Two Months

def compare_rentals(df1, df2, month1, year1, month2, year2):
    # Merge the two DataFrames on customer_id
    merged_df = pd.merge(df1, df2, on='customer_id', how='inner')
    
    # Calculate the difference in rentals between the two months
    merged_df['difference'] = merged_df[f'rentals_{month2}_{year2}'] - merged_df[f'rentals_{month1}_{year1}']
    
    return merged_df

# Using the functions:

# Get rental data for May and June
may_data = rentals_month(engine, 5, 2020)
june_data = rentals_month(engine, 6, 2020)

# Count rentals by customer for May and June
may_counts = rental_count_month(may_data, 5, 2020)
june_counts = rental_count_month(june_data, 6, 2020)

# Compare the rentals between May and June
comparison = compare_rentals(may_counts, june_counts, 5, 2020, 6, 2020)
print(comparison)


Empty DataFrame
Columns: [rentals_5_2020, customer_id, rentals_6_2020, difference]
Index: []
